In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_cohere import ChatCohere
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [2]:
model = ChatCohere(model="command-r")

# Making the model _stateful_

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
store = {} # Will store the messages against sessions id

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {
    "configurable": {
        "session_id": "a2b3c",
    },
}

response = with_message_history.invoke(
    [HumanMessage("Hi! I'm Mark.")],
    config=config
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage("What's my name?")],
    config=config
)

response.content

In [ ]:
store

In [ ]:
response = with_message_history.invoke(
    [HumanMessage("What's my name?")],
    config={"configurable":{"session_id": "e2f3g"}} # Passing a different config
)
response.content

In [ ]:
store

# Persisting state across runs

## Session factory

In [3]:
from pathlib import Path
import re
from typing import Callable, Union
from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory


def _is_valid_identifier(value: str) -> bool:
    """Check if the value is a valid identifier."""
    # Use a regular expression to match the allowed characters
    valid_characters = re.compile(r"^[a-zA-Z0-9-_]+$")
    return bool(valid_characters.match(value))

def create_session_factory(
    base_dir: Union[str, Path],
) -> Callable[[str], BaseChatMessageHistory]:
    """Create a factory that can retrieve chat histories.

    The chat histories are keyed by user ID and conversation ID.

    Args:
        base_dir: Base directory to use for storing the chat histories.

    Returns:
        A factory that can retrieve chat histories keyed by user ID and conversation ID.
    """
    base_dir_ = Path(base_dir) if isinstance(base_dir, str) else base_dir
    if not base_dir_.exists():
        base_dir_.mkdir(parents=True)

    def get_chat_history(user_id: str, conversation_id: str) -> FileChatMessageHistory:
        """Get a chat history from a user id and conversation id."""
        if not _is_valid_identifier(user_id):
            raise ValueError(
                f"User ID {user_id} is not in a valid format. "
                "User ID must only contain alphanumeric characters, "
                "hyphens, and underscores."
                "Please include a valid cookie in the request headers called 'user-id'."
            )
        if not _is_valid_identifier(conversation_id):
            raise ValueError(
                f"Conversation ID {conversation_id} is not in a valid format. "
                "Conversation ID must only contain alphanumeric characters, "
                "hyphens, and underscores. Please provide a valid conversation id "
                "via config. For example, "
                "chain.invoke(.., {'configurable': {'conversation_id': '123'}})"
            )

        user_dir = base_dir_ / user_id
        if not user_dir.exists():
            user_dir.mkdir(parents=True)
        file_path = user_dir / f"{conversation_id}.json"
        return FileChatMessageHistory(str(file_path))

    return get_chat_history

In [4]:
from typing import TypedDict


class InputChat(TypedDict):
    """Input for the chat endpoint."""
    human_input: str
    """Human input"""
    history: list

In [5]:
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant by the name of Bob."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{human_input}"),
    ]
)

chain = prompt | model | StrOutputParser()

chain_with_history = RunnableWithMessageHistory(
    chain,
    create_session_factory("chat_histories"),
    input_messages_key="human_input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
).with_types(input_type=InputChat)

In [6]:
config = {
    "user_id": "mark",
    "conversation_id": "abc",
}

def talk(text: str):
    response = chain_with_history.invoke(
        input=InputChat(human_input=text),
        config=config
    )
    return response

In [7]:
talk("Hello, my name's Mark")

"Hi Mark! It's nice to meet you. How are you doing today?"

In [8]:
talk("Good! Thanks for asking. Wbu?")

"I'm doing well, thank you for asking! I'm having a great day so far and I'm happy to be of assistance to you. As an AI chatbot, my role is to offer helpful and thorough responses to your queries, so please feel free to ask me anything you'd like. How's your week been so far?"

In [9]:
talk("What's the opposite of top?")

'The opposite of "top" would be "bottom." They are opposites in the sense of position or direction. While "top" refers to the uppermost part or highest point, "bottom" represents the lowest position or the base. \n\nThe words "top" and "bottom" are often used to describe the spatial relationships of objects or to indicate relative positions in three-dimensional space.'